In [1]:
using HDF5
using Printf
using DataFrames
using QAOA

In [2]:
function energies_and_bitstrings_qaoa(annealing_problem)
    L = annealing_problem.num_qubits
    h = annealing_problem.local_fields
    J = annealing_problem.couplings
    
    bit_string_df = DataFrame( bit_string = [], energy = Float64[]);
    
    bitstrings = [string(i, base=2, pad=L) |> reverse for i in 0:(2^L - 1)]
    bitvals = [parse.(Int, [bitstring[j] for j in 1:L]) for bitstring in bitstrings]
    spins = [1 .- 2s for s in bitvals]
    
    for spin in spins
        energy = sum([-h[l] * spin[l] for l in 1:L]) + sum([-J[i, j] * spin[i] * spin[j] for i in 1:L for j in (i+1):L])
        push!(bit_string_df,[ spin, energy])
    end
    
    return bit_string_df
end

energies_and_bitstrings_qaoa (generic function with 1 method)

In [3]:
"Build SK model graph from J, h"
function sk_instance_to_graph(J::Matrix{Float64}, h::Vector{Float64})
    N = length(h)
    edge_src = Int[]
    edge_dst = Int[]
    edge_attr = Float32[]
    
    for i in 1:N, j in 1:N
        if i != j
            push!(edge_src, i)
            push!(edge_dst, j)
            push!(edge_attr, Float32(J[i, j]))
        end
    end

    # Node features: h_i, mean(|J|), std(|J|)
    node_features = zeros(Float32, N, 3)
    for i in 1:N
        J_i = abs.(J[i, setdiff(1:N, i)])
        node_features[i, :] = Float32.([h[i], mean(J_i), std(J_i)])
    end

    g = SimpleDiGraph(N)
    for (s, d) in zip(edge_src, edge_dst)
        add_edge!(g, s, d)
    end

    return (
        g,                        # Graph structure
        node_features,           # N × d node features
        edge_attr,               # E vector (one per edge)
        edge_src, edge_dst       # source and target indices
    )
end

sk_instance_to_graph

In [4]:
function save_instance_with_solution(file_path::String, n::Int, output_dir::String = ".")
    # Extract seed from file name
    seed_match = match(r"seed_(\d+)", file_path)
    seed = seed_match !== nothing ? parse(Int, seed_match.captures[1]) : error("Seed not found")

    # Load J and h
    gs_energy = h5read(file_name, "ground_state_energy") 
    J = h5read(file_path, "couplings")
    h = h5read(file_path, "local_fields")
    mf_problem = Problem(0, h, J);

    # Compute solution
    z_opt = energies_and_bitstrings_qaoa(mf_problem)

    # Create new file name
    file_name = splitext(basename(file_path))[1]
    new_file = joinpath(output_dir, "$(file_name)_solution.h5")

    # Write to new h5 file
    h5write(new_file, "couplings", J)
    h5write(new_file, "local_fields", h)
    h5write(new_file, "ground_state_energy", gs_energy)
    h5write(new_file, "solution_bitstring", z_opt)
    #h5write(new_file, "seed", seed)

    println("Saved: $new_file")
    return new_file
end

save_instance_with_solution (generic function with 2 methods)

SK data

In [11]:
# number of spins
N = 18
#N = parse(Int, ARGS[1])

PATH = raw"/home/ubuntu/aqc_QAOA/SpinFluctuations.jl"
subdir = "small_gaps"

folder_name = PATH * @sprintf("//data//N_%i//", N);

### change N in the pattern individually 
pattern = r"hard_random_SK_instance_N_18_seed_(\d+)\.h5"
###

instance_names = readdir(folder_name)
loop_var = 1
total_num_inst = 0


for (k, instance_name) in enumerate(instance_names[loop_var+0:loop_var+109])

    seed = match(pattern, instance_name)[1]
    seed = parse(Int64, seed)
    #println("seed: ",seed)
    
    #spin_idx = 2
    file_name = folder_name * @sprintf("hard_random_SK_instance_N_%i_seed_%i.h5", N , seed)
    
    gs_energy = h5read(file_name, "ground_state_energy") 
    J = h5read(file_name, "couplings")
    h = h5read(file_name, "local_fields")
    mf_problem = Problem(0, h, J);

    # Compute solution
    z_opt = energies_and_bitstrings_qaoa(mf_problem)

    sorted_df = sort(z_opt, :energy)

    lowest_energy_row = sorted_df[1, :]

    z_opt = lowest_energy_row[1]


    PATH_w = raw"/home/ubuntu/aqc_GNN/"
    #subdir = "small_gaps"

    folder_name_w = PATH_w * @sprintf("//SK_data//N_%i//", N);

    h5open(folder_name_w * @sprintf("hard_SK_instance_bitstring_N_%i_seed_%i.h5", N, seed), "w") do file
        write(file, "couplings", J)            
        write(file, "local_fields", h)         
        write(file, "ground_state_energy", gs_energy)        
        write(file, "solution_bitstring", z_opt)
    
    end
    #println(z_opt)

end    

MAX2SAT

In [ ]:
# number of spins
N = 16
num_clauses = 3N

PATH = raw"/home/ubuntu/MAX2SAT/MAX2SATQuantumData/Mirkarimi_data/"
#subdir = "small_gaps"

#folder_name = PATH * @sprintf("/HDF5/N_", N);
println(PATH)

instance_names = collect(1:2000)
loop_var = 1
total_num_inst = 0


for (k, idx) in enumerate(instance_names[1:1000])
    
    #spin_idx = 2
 
    file_name = PATH * @sprintf("/HDF5/max2sat_typical_instance_%04i_from_arxiv_2206_06876_N_%i_num_clauses_%i.h5", idx, N, num_clauses)
  
    #gs_energy = h5read(file_name, "ground_state_energy") 
    J = h5read(file_name, "coupling_matrix")
    h = h5read(file_name, "local_fields")
    h = - h # MAX2SAT minus sign added
    mf_problem = Problem(0, h, J);

    # Compute solution
    z_opt = energies_and_bitstrings_qaoa(mf_problem)

    sorted_df = sort(z_opt, :energy)

    lowest_energy_row = sorted_df[1, :]

    z_opt = lowest_energy_row[1]

    gs_energy = lowest_energy_row[2]
    #println(gs_energy)

    PATH_w = raw"/home/ubuntu/aqc_GNN/"
    #subdir = "small_gaps"

    folder_name_w = PATH_w * @sprintf("//MAX2SAT_data//N_%i//", N);

    h5open(folder_name_w * @sprintf("MAX2SAT_instance_bitstring_N_%i_idx_%04i.h5", N, idx), "w") do file
        write(file, "couplings", J)            
        write(file, "local_fields", h)         
        write(file, "ground_state_energy", gs_energy)        
        write(file, "solution_bitstring", z_opt)
    
    end
    #println(z_opt)

end    